# TWAS & Coloc

## 1. TWAS
### 1.1 Transcriptome-Wide Association Study (TWAS)

**TWAS** is a method used to identify genes associated with complex traits by leveraging predicted gene expression levels. It combines precomputed molecular phenotype prediction weights with GWAS summary statistics to perform gene-trait association tests. 


#### 1.2 Input File Formats

- GWAS Metadata (`--gwas_meta_data`)    
A TSV file specifying study ID, chromosome, path to summary statistics, and a column mapping file.  
- LD Reference Metadata (`--ld_meta_data`)    
A TSV file with chromosome, start and end positions, and paths to LD matrix files.  
- xQTL Weight Metadata (`--xqtl_meta_data`)    
Metadata file containing gene region coordinates and paths to weight files stored in RDS format.  
- Genomic Region Definitions (`--regions`)    
A BED file defining analysis regions.  
- xQTL Type Table (`--xqtl_type_table`)    
A mapping file defining the molecular phenotype types and their associated biological context.


#### 1.3 Key Parameters

- `--rsq_cutoff 0.01`: Cross-validation R² threshold used to determine whether a gene is predictable  
- `--rsq_pval_cutoff 0.05`: Cross-validation p-value threshold  


#### 1.4 Analysis Workflow  

1). **Weight Loading**: Load TWAS prediction weights from RDS files.     
2). **Predictable Gene Identification**: Determine which genes are reliably predictable based on cross-validation performance.    
3). **TWAS Testing**: Compute TWAS Z-scores by combining predicted expression with GWAS statistics.    
4). **Mendelian Randomization (MR)**: Perform MR analysis on candidate genes.    



In [ ]:
# gwas input 
cd /home/ubuntu/xqtl_protocol_exercise
head data/twas/gwas_meta_test.tsv

study_id	chrom	file_path	column_mapping_file
Bellenguez_2022	11	data/twas/RSS_QC_RAISS_imputed.AD_Bellenguez_2022_April9_chr11.tsv.gz	data/twas/Bellenguez.yml


In [ ]:
# ld meta file
head reference_data/ADSP_R4_EUR/ld_meta_file.tsv

#chrom	start	end	path
chr1	101384274	104443097	chr1/chr1_101384274_104443097.cor.xz,chr1/chr1_101384274_104443097.cor.xz.bim
chr1	104443097	106225286	chr1/chr1_104443097_106225286.cor.xz,chr1/chr1_104443097_106225286.cor.xz.bim
chr1	106225286	109761915	chr1/chr1_106225286_109761915.cor.xz,chr1/chr1_106225286_109761915.cor.xz.bim
chr1	109761915	111483530	chr1/chr1_109761915_111483530.cor.xz,chr1/chr1_109761915_111483530.cor.xz.bim
chr1	111483530	113276642	chr1/chr1_111483530_113276642.cor.xz,chr1/chr1_111483530_113276642.cor.xz.bim
chr1	113276642	115338054	chr1/chr1_113276642_115338054.cor.xz,chr1/chr1_113276642_115338054.cor.xz.bim
chr1	11328222	12710318	chr1/chr1_11328222_12710318.cor.xz,chr1/chr1_11328222_12710318.cor.xz.bim
chr1	115338054	117562321	chr1/chr1_115338054_117562321.cor.xz,chr1/chr1_115338054_117562321.cor.xz.bim
chr1	117562321	119804207	chr1/chr1_117562321_119804207.cor.xz,chr1/chr1_117562321_119804207.cor.xz.bim


In [ ]:
# twas weight file
head data/twas/mwe_twas_pipeline_test_small.tsv

"#chr" "region_id" "TSS" "start" "end" "contexts" "original_data"
"chr11" "ENSG00000073921" 86069881 84957175 87360000 NA "data/twas/ROSMAP_DeJager.chr11_ENSG00000073921.univariate_twas_weights.rds,data/twas/ROSMAP_DeJager.chr11_ENSG00000073921.multicontext_twas_weights.rds"


To perform TWAS, you need to generate TWAS weight first.   
see: 
- exercise notebook: `data/exercise_notebook/qtl_association_finemapping/2_finemapping.ipynb`
- pipeline: `pipeline/mnm_regression.ipynb susie_twas`

In [ ]:
# change this kernel to R code
# TWAS weights structure
setwd('/home/ubuntu/xqtl_protocol_exercise')
twas_weight = readRDS('data/twas/ROSMAP_DeJager.chr11_ENSG00000002330.univariate_twas_weights.rds')
str(twas_weight)

List of 1
 $ ENSG00000002330:List of 10
  ..$ AC_DeJager_eQTL_ENSG00000002330                :List of 7
  .. ..$ susie_weights_intermediate:List of 4
  .. .. ..$ mu                    : num [1:8, 1:7801] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. .. ..$ : NULL
  .. .. .. .. ..$ : chr [1:7801] "chr11:61201723:C:T" "chr11:61202327:A:G" "chr11:61202458:A:G" "chr11:61202571:G:C" ...
  .. .. ..$ lbf_variable          : num [1:8, 1:7801] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. .. ..$ : NULL
  .. .. .. .. ..$ : chr [1:7801] "chr11:61201723:C:T" "chr11:61202327:A:G" "chr11:61202458:A:G" "chr11:61202571:G:C" ...
  .. .. ..$ X_column_scale_factors: Named num [1:7801] 0.184 0.632 0.636 0.637 0.614 ...
  .. .. .. ..- attr(*, "names")= chr [1:7801] "chr11:61201723:C:T" "chr11:61202327:A:G" "chr11:61202458:A:G" "chr11:61202571:G:C" ...
  .. .. ..$ pip                   : Named num [1:7801] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..

In [ ]:
# r code
twas_weight_by_method = twas_weight$ENSG00000002330$AC_DeJager_eQTL_ENSG00000002330$twas_weights
str(twas_weight_by_method)

List of 6
 $ enet_weights   : num [1:7801, 1] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:7801] "chr11:61201723:C:T" "chr11:61202327:A:G" "chr11:61202458:A:G" "chr11:61202571:G:C" ...
  .. ..$ : NULL
 $ lasso_weights  : num [1:7801, 1] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:7801] "chr11:61201723:C:T" "chr11:61202327:A:G" "chr11:61202458:A:G" "chr11:61202571:G:C" ...
  .. ..$ : NULL
 $ bayes_r_weights: num [1:7801, 1] -1.69e-05 -1.04e-04 -1.65e-04 -1.48e-04 -9.45e-05 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:7801] "chr11:61201723:C:T" "chr11:61202327:A:G" "chr11:61202458:A:G" "chr11:61202571:G:C" ...
  .. ..$ : NULL
 $ bayes_l_weights: num [1:7801, 1] 8.28e-05 -4.19e-04 -4.27e-04 -6.72e-04 -5.37e-04 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:7801] "chr11:61201723:C:T" "chr11:61202327:A:G" "chr11:61202458:A:G" "chr11:61202571:G:C" ...
  .. ..$ : NULL
 $ mrash_weights  : num [1:7801, 1]

In [ ]:
cd /home/ubuntu/xqtl_protocol_exercise
sos run pipeline/twas_ctwas.ipynb twas \
   --cwd output/twas --name test \
   --gwas_meta_data data/twas/gwas_meta_test.tsv \
   --ld_meta_data reference_data/ADSP_R4_EUR/ld_meta_file.tsv \
   --regions data/twas/EUR_LD_blocks.bed \
   --xqtl_meta_data data/twas/mwe_twas_pipeline_test_small.tsv \
   --xqtl_type_table data/twas/data_type_table.txt \
   --rsq_pval_cutoff 0.05 --rsq_cutoff 0.01    

/home/al4225/.pixi/envs/python/lib/python3.12/site-packages/sos/targets.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.
INFO: Running get_analysis_regions: 
Skipping 1360 out of 1361 regions, no overlapping xQTL weights found. 
INFO: get_analysis_regions is completed.
INFO: get_analysis_regions output:   filtered_regional_xqtl_files
INFO: Running twas: 
INFO: twas is completed.
INFO: twas output:   /mnt/vast/hpc/homes/al4225/xqtl_protocol_data/output/twas/twas/test.chr11_84267999_86714492.twas.tsv.gz /mnt/vast/hpc/homes/al4225/xqtl_protocol_data/output/twas/twas/test.chr11_84267999_86714492.twas_data.rds... (3 items)
INFO: W

In [ ]:
## twas results
cd /home/ubuntu/xqtl_protocol_exercise
zcat output/twas/twas/test.chr11_84267999_86714492.twas.tsv.gz | head -n 4

chr	molecular_id	TSS	start	end	context	gwas_study	method	is_imputable	is_selected_method	rsq_cv	pval_cv	twas_z	twas_pval	type	block
11	ENSG00000073921	86069881	84957175	87360000	AC_DeJager_eQTL	Bellenguez_2022	bayes_l	TRUE	FALSE	0.00994563755691413	0.0151210816406667	-3.01085890276408	0.00260509875533349	eQTL	chr11_84267999_86714492
11	ENSG00000073921	86069881	84957175	87360000	AC_DeJager_eQTL	Bellenguez_2022	bayes_r	TRUE	FALSE	0.0129222135582411	0.00558266481673196	-3.60984545831109	0.000306379488103016	eQTL	chr11_84267999_86714492
11	ENSG00000073921	86069881	84957175	87360000	AC_DeJager_eQTL	Bellenguez_2022	enet	TRUE	TRUE	0.0302273357404713	2.06909070605428e-05	-3.66968282001104	0.000242851605595364	eQTL	chr11_84267999_86714492


In [ ]:
## mr results
zcat output/twas/twas/test.chr11_84267999_86714492.mr_result.tsv.gz | head

gene_name	num_CS	num_IV	cpip	meta_eff	se_meta_eff	meta_pval	Q	Q_pval	I2	context	gwas_study
ENSG00000073921										AC_DeJager_eQTL	Bellenguez_2022
ENSG00000073921										Ast_DeJager_eQTL	Bellenguez_2022
ENSG00000073921										DLPFC_Bennett_pQTL_chr11_Q13492	Bellenguez_2022
ENSG00000073921										DLPFC_DeJager_eQTL	Bellenguez_2022
ENSG00000073921	2	113	0.951	-0.002	0.001	0	1.432	0.231	0.302	Exc_DeJager_eQTL	Bellenguez_2022
ENSG00000073921										Inh_DeJager_eQTL	Bellenguez_2022
ENSG00000073921	2	8	0.954	-0.002	0	0	0.132	0.717	0	Mic_DeJager_eQTL	Bellenguez_2022
ENSG00000073921										OPC_DeJager_eQTL	Bellenguez_2022
ENSG00000073921	1	4	0.985	0.002	0	0	0	0	0	Oli_DeJager_eQTL	Bellenguez_2022


## 2. coloc

### 2.1 global enrichment analysis for the calculation of enrichment parameters between xQTL and GWAS data
This command executes the **`xqtl_gwas_enrichment`** workflow, which analyzes enrichment relationships between xQTL and GWAS data

#### **Purpose**

The purpose of this step is to calculate enrichment parameters (a0, a1) between xQTL and GWAS data and generate prior probabilities (p1, p2, p12) for subsequent colocalization analysi

#### **Method**

The workflow uses the **`xqtl_enrichment_wrapper`** function to perform enrichment analysis . The method:

1. Identifies GWAS blocks with top loci using single variant regression methods
2. Maps analysis regions to overlapping gene regions with corresponding QTL files containing top loci tables
3. Finds contexts within xQTL metadata that include top loci results for each gene
4. Executes enrichment analysis to generate parameters SuSiE_enloc.ipynb:102-106

#### **Input Data**

**Metadata files:**

- **`-gwas-meta-data`**: Metadata file for GWAS fine-mapping results
- **`-xqtl-meta-data`**: Metadata file for xQTL fine-mapping results
- **`-context_meta`**: Meta file showing analysis names and contained contexts

**Data paths:**

- **`-qtl-path`** and **`-gwas_path`**: Directory paths for original fine-mapping results

#### **Parameter Interpretation**

**Object access parameters:**

- **`-xqtl_finemapping_obj`**: Table name in xQTL RDS files to get fine-mapping results
- **`-xqtl_varname_obj`**: Table name to get variable names
- **`-gwas_finemapping_obj`** and **`-gwas_varname_obj`**: Corresponding parameters for GWAS data
- **`-xqtl_region_obj`**: Table name to get region information

#### **Output**

Generates enrichment analysis result files: **`*.enrichment.rds`**, containing global enrichment estimates that combine all input data for each context. The output file path format is: **`{cwd}/{name}.{context}.enrichment.rds`** 

#### **Relationship to Downstream Analysis**

This enrichment analysis is a prerequisite step for colocalization analysis. The generated enrichment parameters serve as prior probability inputs for the **`susie_coloc`** workflow. In colocalization analysis, the system automatically reads enrichment result files to set p1, p2, and p12 prior probabilities.

#### **Notes**

This workflow is the first stage of the SuSiE-enloc framework, specifically handling enrichment analysis between fine-mapping results from different genomic regions. The workflow automatically handles region overlap identification and context matching, providing statistically sound prior probabilities for subsequent colocalization analysis.

In [ ]:
sos run pipeline/SuSiE_enloc.ipynb xqtl_gwas_enrichment    \
    --gwas-meta-data data/susie_enloc_data/demo_gwas.block_results_db.tsv \
    --xqtl-meta-data data/susie_enloc_data/demo_overlap.overlapped.gwas.tsv \
    --xqtl_finemapping_obj preset_variants_result susie_result_trimmed              \
    --xqtl_varname_obj preset_variants_result variant_names             \
    --gwas_finemapping_obj AD_Bellenguez_2022 RSS_QC_RAISS_imputed susie_result_trimmed              \
    --gwas_varname_obj  AD_Bellenguez_2022 RSS_QC_RAISS_imputed variant_names             \
    --xqtl_region_obj  region_info   grange       \
    --qtl-path data/susie_enloc_data \
    --gwas_path data/susie_enloc_data \
    --context_meta data/susie_enloc_data/context_meta.tsv \
    --cwd output/xqtl_gwas_enrichment


/home/al4225/.pixi/envs/python/lib/python3.12/site-packages/sos/targets.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.
INFO: Running get_analysis_regions: get overlapped regions by gene and block region for enrichment analysis
INFO: get_analysis_regions is completed.
INFO: get_analysis_regions output:   regional_data
INFO: Running xqtl_gwas_enrichment: 
INFO: xqtl_gwas_enrichment is completed.
INFO: xqtl_gwas_enrichment output:   /mnt/vast/hpc/homes/al4225/xqtl_protocol_data/output/xqtl_gwas_enrichment/demo_overlap.overlapped.demo_gwas.Knight_eQTL_brain.enrichment.rds
INFO: Workflow xqtl_gwas_enrichment (ID=w163a9b2b63d2f22

In [ ]:
# ouput of enrichment parameter between xqtl and gwas
# R COED
setwd('/home/ubuntu/xqtl_protocol_exercise')
coloc_enrichment = readRDS('output/xqtl_gwas_enrichment/demo_overlap.overlapped.demo_gwas.Knight_eQTL_brain.enrichment.rds')
str(coloc_enrichment)

List of 2
 $                     :List of 10
  ..$ Alternative (coloc) p1   : num 0.000558
  ..$ Alternative (coloc) p12  : num 6.31e-08
  ..$ Alternative (coloc) p2   : num 0.000113
  ..$ Effective MI rounds      : num 24
  ..$ Enrichment (no shrinkage): num -0.409
  ..$ Enrichment (w/ shrinkage): num -0.000151
  ..$ Intercept                : num -7.49
  ..$ sd (intercept)           : num 0.316
  ..$ sd (no shrinkage)        : num 52
  ..$ sd (w/ shrinkage)        : num 1
 $ unused_xqtl_variants:List of 1
  ..$ : chr [1:18] "1:20568238:C:T" "1:20621145:G:A" "1:21427833:C:G" "1:21688743:C:T" ...


### 2.2 pairwise colocalization analysis to identify shared causal variants between xQTL and GWAS
This command executes the **`susie_coloc`** workflow, which performs colocalization analysis between xQTL and GWAS data to identify shared causal variants.

#### **Purpose**

The purpose of this step is to perform pairwise colocalization analysis between xQTL and GWAS fine-mapping results to determine whether observed associations share causal variants in overlapping genomic regions. This analysis identifies contexts with top loci results for each gene and applies **`susie_coloc`** to analyze each gene under each identified condition.

#### **Method**

The workflow uses the **`coloc_wrapper`** function to perform colocalization analysis. The method:

1. **Region Overlap Detection**: Identifies GWAS blocks that contain overlapping top loci variants for each gene using genomic coordinate matching
2. **Prior Probability Setting**: Either loads enrichment-derived priors from previous analysis or uses default values when **`-skip-enrich`** is specified
3. **Colocalization Analysis**: Applies **`coloc_wrapper`** for each xQTL-GWAS file pair with appropriate prior probabilities

#### **Input Data**

**Metadata files:**

- **`-gwas-meta-data`**: GWAS fine-mapping results metadata
- **`-xqtl-meta-data`**: xQTL fine-mapping results metadata with overlap information
- **`-context_meta`**: Analysis names and context mappings

**Data paths:**

- **`-qtl-path`** and **`-gwas_path`**: Directory paths for original fine-mapping results

#### **Parameter Interpretation**

**Object access parameters:**

- **`-xqtl_finemapping_obj`**: Table name in xQTL RDS files for fine-mapping results
- **`-xqtl_varname_obj`**: Table name for variable names
- **`-gwas_finemapping_obj`** and **`-gwas_varname_obj`**: Corresponding GWAS parameters
- **`-xqtl_region_obj`**: Table name for region information

**Analysis control parameters:**

- **`-skip-enrich`**: Skips enrichment analysis and uses default prior probabilities (p1=1e-4, p2=1e-4, p12=5e-6)
- **`-ld_meta_file_path`**: LD reference metadata for post-processing credible sets

#### **Output**

Generates colocalization result files: **`*.coloc.rds`** containing colocalization statistics and posterior probabilities for each gene-context pair. When LD metadata is provided, also outputs credible set files (**`*.coloc_res`**) with variant-level results SuSiE_enloc.ipynb:851 .

#### **Notes**

This workflow represents the second stage of the SuSiE-enloc framework, performing gene-by-gene colocalization analysis on regions identified to have overlapping variants. The **`--skip-enrich`** parameter allows bypassing enrichment analysis when using default priors, making the analysis faster but potentially less statistically informed. The inclusion of LD metadata enables variant-level credible set reporting for significant colocalization results.

In [ ]:
# gwas meta
cd /home/ubuntu/xqtl_protocol_exercise
head data/susie_enloc_data/demo_gwas.block_results_db.tsv

#chr	start	end	region_id	TSS	original_data	combined_data	combined_data_sumstats	conditions	conditions_top_loci	conditions_top_loci_minp
chr1	17351816	20110062	chr1_17351816_20110062	NA	RSS_QC_RAISS_imputed.chr1_17351816_20110062.univariate_susie_rss.rds	demo_gwas.chr1_17351816_20110062.cis_results_db.export.rds	demo_gwas.chr1_17351816_20110062.cis_results_db.export_sumstats.rds	AD_Bellenguez_2022,AD_Jansen_2021,AD_Kunkle_Stage1_2019,AD_Wightman_Full_2021,AD_Wightman_Excluding23andMe_2021,AD_Wightman_ExcludingUKBand23andME_2021,AD_Bellenguez_EADB_2022,AD_Bellenguez_EADI_2022	AD_Bellenguez_2022,AD_Kunkle_Stage1_2019,AD_Bellenguez_EADB_2022	NA
chr1	20110062	22020160	chr1_20110062_22020160	NA	RSS_QC_RAISS_imputed.chr1_20110062_22020160.univariate_susie_rss.rds	demo_gwas.chr1_20110062_22020160.cis_results_db.export.rds	demo_gwas.chr1_20110062_22020160.cis_results_db.export_sumstats.rds	AD_Bellenguez_2022,AD_Jansen_2021,AD_Kunkle_Stage1_2019,AD_Wightman_Full_2021,AD_Wightman_Excluding23andMe

In [ ]:
sos run pipeline/SuSiE_enloc.ipynb susie_coloc \
    --gwas-meta-data data/susie_enloc_data/demo_gwas.block_results_db.tsv \
    --xqtl-meta-data data/susie_enloc_data/demo_overlap.overlapped.gwas.tsv \
    --xqtl_finemapping_obj preset_variants_result susie_result_trimmed \
    --xqtl_varname_obj preset_variants_result variant_names \
    --gwas_finemapping_obj AD_Bellenguez_2022 RSS_QC_RAISS_imputed susie_result_trimmed \
    --gwas_varname_obj  AD_Bellenguez_2022 RSS_QC_RAISS_imputed variant_names \
    --xqtl_region_obj  region_info grange \
    --qtl-path data/susie_enloc_data \
    --skip-enrich \
    --gwas_path data/susie_enloc_data \
    --context_meta data/susie_enloc_data/context_meta.tsv \
    --ld_meta_file_path data/ld_meta_file.tsv \
    --cwd output/susie_coloc

/home/al4225/.pixi/envs/python/lib/python3.12/site-packages/sos/targets.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.
INFO: Running get_overlapped_analysis_regions: get overlapped regions from flatten table, to get the regions with overlapped variants and select those regions for coloc analysis
INFO: get_overlapped_analysis_regions is completed.
INFO: get_overlapped_analysis_regions output:   overlapped_regional_data
INFO: Running susie_coloc: 
INFO: susie_coloc is completed.
INFO: susie_coloc output:   /mnt/vast/hpc/homes/al4225/xqtl_protocol_data/output/susie_coloc/susie_coloc/demo_overlap.overlapped.demo_gwas.Knight_eQT

In [ ]:
# r code
# coloc results
setwd('/home/ubuntu/xqtl_protocol_exercise')
coloc_result = readRDS('output/susie_coloc/susie_coloc/demo_overlap.overlapped.demo_gwas.Knight_eQTL_brain@ENSG00000142798.coloc.rds')
str(coloc_result)

List of 1
 $ ENSG00000142798:List of 5
  ..$ summary        :Classes ‘data.table’ and 'data.frame':	10 obs. of  10 variables:
  .. ..$ nsnps    : int [1:10] 8826 8826 8826 8826 8826 8826 8826 8826 8826 8826
  .. ..$ hit1     : chr [1:10] "1:21912282:C:G" "1:21912282:C:G" "1:21912282:C:G" "1:21912282:C:G" ...
  .. ..$ hit2     : chr [1:10] "1:23996556:C:T" "1:22187644:C:A" "1:22187644:C:A" "1:22187644:C:A" ...
  .. ..$ PP.H0.abf: num [1:10] 0.0165 0.0716 0.0715 0.0715 0.0715 ...
  .. ..$ PP.H1.abf: num [1:10] 0.102 0.443 0.443 0.443 0.443 ...
  .. ..$ PP.H2.abf: num [1:10] 0.1217 0.0645 0.0645 0.0645 0.0645 ...
  .. ..$ PP.H3.abf: num [1:10] 0.753 0.399 0.399 0.399 0.399 ...
  .. ..$ PP.H4.abf: num [1:10] 0.00699 0.02221 0.0222 0.0222 0.0222 ...
  .. ..$ idx1     : int [1:10] 1 1 1 1 1 1 1 1 1 1
  .. ..$ idx2     : int [1:10] 1 2 3 4 5 6 7 8 9 10
  .. ..- attr(*, ".internal.selfref")=<externalptr> 
  ..$ results        :Classes ‘data.table’ and 'data.frame':	8826 obs. of  11 variables:


#### Interpretation of `coloc` Analysis Results (Gene: `ENSG00000142798`)

##### 1). Overall Structure

The provided object is a nested list in R, structured specifically for interpreting coloc analysis results. It contains five main components:

- `summary`: Posterior probabilities for different coloc hypotheses.
- `results`: SNP-level posterior probabilities (PP.H4).
- `priors`: Priors used in Bayesian coloc analysis.
- `analysis_region`: Genomic region analyzed.
- `sets`: Credible sets; here `NULL`, indicating no credible set was formed.

---

##### 2). `summary` Table

This table contains posterior probabilities for five coloc hypotheses across SNP pairs:

- `nsnps`: Number of SNPs analyzed (8826).
- `hit1`: Representative SNP for signal 1.
- `hit2`: Representative SNP for signal 2.

Five coloc hypotheses probabilities:

- `PP.H0.abf`: Neither trait has an association (null hypothesis).
- `PP.H1.abf`: Only trait 1 is associated.
- `PP.H2.abf`: Only trait 2 is associated.
- `PP.H3.abf`: Both traits are associated, but at different SNPs.
- `PP.H4.abf`: Both traits are associated at the same SNP (colocalization).

Higher `PP.H4.abf` indicates stronger evidence for colocalization.

---

##### 3). `results` Table

Contains posterior probabilities for each of the 8826 SNPs across ten coloc scenarios (`rows`):

- `snp`: SNP identifier (e.g., `"1:20520243:G:A"`).
- `SNP.PP.H4.row1` to `SNP.PP.H4.row10`: SNP-specific posterior probabilities for colocalization scenarios corresponding to each row in `summary`.

High SNP-level posterior probabilities indicate likely colocalization positions.

---

##### 4). `priors`

Prior probabilities used in Bayesian analysis:

- `p1`: Prior probability that trait 1 is associated.
- `p2`: Prior probability that trait 2 is associated.
- `p12`: Prior probability of colocalization between trait 1 and trait 2.

Typically set low, reflecting conservative assumptions.

---

##### 5). `analysis_region`

Genomic region analyzed (e.g., `"chr1:20520000-24080000"`).

---

##### 6). `sets`

Contains credible sets (`cs`):

- Here `cs` is `NULL`, indicating no credible set formed, possibly due to weak evidence.

---

##### 7). Guidelines for Interpretation

Interpretation of `PP.H4.abf`:

- `PP.H4.abf > 0.75`: Strong evidence of colocalization.
- `0.25 ≤ PP.H4.abf ≤ 0.75`: Moderate evidence of colocalization.
- `PP.H4.abf < 0.25`: Weak evidence of colocalization.

When `PP.H4.abf` is high, further examine individual SNP-level probabilities (`results`) to pinpoint the SNP responsible for the shared signal.

---

##### 8). Practical Example

For row 2 of the `summary`:

- `hit1`: `1:21912282:C:G`
- `hit2`: `1:22187644:C:A`
- `PP.H4.abf`: 0.02221 (~2.2%)

The low `PP.H4.abf` indicates weak colocalization evidence. The higher `PP.H3.abf` (~40%) implies distinct SNP associations.

Inspect `results` for high individual SNP posterior probabilities (e.g., > 0.9) to identify candidate colocalized SNPs.

---

##### 9). Summary of Key Points

- Inspect `summary` for overall colocalization evidence (`PP.H4.abf`).
- Check `results` for SNP-level posterior probabilities to identify potential colocalized SNPs.
- A `NULL` credible set indicates insufficient evidence to confidently define a credible SNP set.

### 2.3 ColocBoost

**ColocBoost** is a multi-trait colocalization analysis tool designed to identify shared genetic variants influencing multiple molecular traits.


### Core Features

- Performs colocalization analysis within genomic regions accounting for **multiple causal variants**
- Scales to the analysis of **hundreds of traits**
- Supports inclusion or exclusion of **GWAS summary statistics**
- Requires **individual-level xQTL data from the same cohort**

### Analysis Modes

ColocBoost supports three major analysis modes:

- xQTL-only Analysis (`--xqtl-coloc`):  
  Performs colocalization only among molecular traits.

- Joint GWAS Analysis (`--joint-gwas`):  
  Integrates all GWAS traits in a combined colocalization analysis.

- Separate GWAS Analysis (`--separate-gwas`):  
  Performs colocalization analysis separately for each GWAS trait.

### `--separate-gwas` vs `--no-separate-gwas`

#### `--separate-gwas` (Enabled by default)

When this flag is active, ColocBoost performs **individual colocalization analyses** for each GWAS trait.

- Generates **one output file per GWAS study**
- Output format: `{base_filename}.cb_xqtl_{study_name}.rds`
- Supports **trait-specific** analysis and result inspection

#### `--no-separate-gwas`

When this flag is used, separate GWAS analyses are **not performed**.

- Skips individual trait analysis
- Reduces computational burden and the number of output files
- Typically used **in combination with** `--joint-gwas`


### Practical Examples

#### xQTL-only Analysis

- Use `--no-separate-gwas` to focus solely on **colocalization between molecular traits**

#### GWAS-xQTL Joint Analysis

- Use `--separate-gwas` to perform **trait-specific** colocalization for each GWAS dataset


In [ ]:
# xQTL-only with --no-separate-gwas parameter
sos run pipeline/colocboost.ipynb colocboost \
    --name test_coloc_boost_xqtl_only  \
    --cwd output/colocboost \
    --genoFile output/plink/wgs.merged.bed \
    --phenoFile output/phenotype/phenotype_by_chrom_for_cis/bulk_rnaseq.phenotype_by_chrom_files.region_list.txt \
    --covFile output/covariate/bulk_rnaseq_tmp_matrix.low_expression_filtered.outlier_removed.tmm.expression.covariates.wgs.merged.plink_qc.plink_qc.prune.pca.Marchenko_PC.gz \
    --customized-association-windows reference_data/TAD/TADB_enhanced_cis.bed \
    --no-separate-gwas --xqtl-coloc \
    --region-name ENSG00000049246 \
    --phenotype-names trait_A

/home/al4225/.pixi/envs/python/lib/python3.12/site-packages/sos/targets.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.
INFO: Running get_rss_analysis_regions: 
INFO: get_rss_analysis_regions is completed.
INFO: get_rss_analysis_regions output:   regional_rss_data
INFO: Running get_analysis_regions: 
Loading customized association analysis window from reference_data/TAD/TADB_enhanced_cis.bed
INFO: get_analysis_regions is completed.
INFO: get_analysis_regions output:   regional_data
INFO: Running colocboost: 
INFO: colocboost is completed.
INFO: colocboost output:   /mnt/vast/hpc/homes/al4225/xqtl_protocol_data/output/colocbo

In [ ]:
# GWAS-xQTL integrated with --separate-gwas parameter
sos run pipeline/colocboost.ipynb colocboost \
    --name colocboost_gwas  \
    --cwd output/colocboost_gwas \
    --genoFile output/plink/wgs.merged.bed \
    --phenoFile output/phenotype/phenotype_by_chrom_for_cis/bulk_rnaseq.phenotype_by_chrom_files.region_list.txt \
    --covFile output/covariate/bulk_rnaseq_tmp_matrix.low_expression_filtered.outlier_removed.tmm.expression.covariates.wgs.merged.plink_qc.plink_qc.prune.pca.Marchenko_PC.gz \
    --customized-association-windows reference_data/TAD/TADB_enhanced_cis.bed \
    --ld-meta-data data/ld_meta_file.tsv \
    --gwas-meta-data data/colocboost/gwas_meta.txt \
    --separate-gwas --xqtl-coloc \
    --region-name ENSG00000049246 \
    --phenotype-names trait_A

/home/al4225/.pixi/envs/python/lib/python3.12/site-packages/sos/targets.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.
INFO: Running get_analysis_regions: 
Loading customized association analysis window from reference_data/TAD/TADB_enhanced_cis.bed
INFO: Running get_rss_analysis_regions: 
Using customized association window file: reference_data/TAD/TADB_enhanced_cis.bed
INFO: get_analysis_regions is completed.
INFO: get_analysis_regions output:   regional_data
INFO: get_rss_analysis_regions is completed.
INFO: get_rss_analysis_regions output:   regional_rss_data
INFO: Running colocboost: 
INFO: colocboost is completed.
INFO

In [ ]:
# output of colocboost
setwd('/home/ubuntu/xqtl_protocol_exercise')
colocboost = readRDS('output/colocboost_gwas/colocboost/colocboost_gwas.chr1_ENSG00000049246.cb_xqtl_Wightman.rds')
str(colocboost)

List of 1
 $ ENSG00000049246:List of 8
  ..$ cos_summary   : NULL
  ..$ vcp           : NULL
  ..$ cos_details   : NULL
  ..$ data_info     :List of 6
  .. ..$ n_outcomes  : int 2
  .. ..$ n_variables : int 6872
  .. ..$ outcome_info:'data.frame':	2 obs. of  4 variables:
  .. .. ..$ outcome_names: chr [1:2] "trait_A_ENSG00000049246" "Wightman"
  .. .. ..$ sample_size  : num [1:2] 150 363646
  .. .. ..$ is_sumstats  : logi [1:2] FALSE TRUE
  .. .. ..$ is_focal     : logi [1:2] FALSE TRUE
  .. ..$ variables   : chr [1:6872] "chr1:6120249:G:A" "chr1:6120384:G:A" "chr1:6120435:G:A" "chr1:6120926:G:A" ...
  .. ..$ coef        :List of 2
  .. .. ..$ trait_A_ENSG00000049246: num [1:6872] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..$ Wightman               : num [1:6872] -1.30e-05 -1.15e-05 -1.18e-05 1.10e-05 -1.31e-05 ...
  .. ..$ z           :List of 2
  .. .. ..$ trait_A_ENSG00000049246: num [1:6872] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..$ Wightman               : num [1:6872] -1.0368 -0.2813 -0.4772 0.0


### Colocboost Output File Structure

- **When `--separate-gwas` is enabled**:  
  ColocBoost outputs **one RDS file per GWAS trait**, containing:
  - Region-level colocalization results
  - Metadata including analysis time, number of SNPs, etc.

- File naming convention follows:  
  `{base_filename}.cb_xqtl_{study_name}.rds`

This output is generated from a ColocBoost analysis examining shared causal variants between xQTL and GWAS data.



The `colocboost` result is a named list of class `"colocboost"`, where each top-level element corresponds to a gene (e.g., `ENSG00000049246`). The internal structure is divided into several components:

---

#### Top-Level Components

| Field | Type | Description |
| --- | --- | --- |
| `cos_summary` | NULL or list | Summary of colocalized signals (empty if none) |
| `vcp` | NULL or list | Variant colocalization probabilities |
| `cos_details` | NULL or list | Details for colocalized signals |
| `data_info` | list | Information on traits, SNPs, z-scores, and coefficients |
| `model_info` | list | Model convergence and log-likelihood tracking |
| `ucos_details` | list | Results for **uncolocalized signals** (unique to colocboost) |
| `region_info` | list | Genomic coordinates and extended window used in analysis |
| `computing_time` | list | Timing for loading, QC, and model fitting steps |

---

#### data_info

| Field | Description |
| --- | --- |
| `n_outcomes` | Number of traits analyzed |
| `n_variables` | Number of SNPs analyzed |
| `outcome_info` | Data frame of each trait’s name, sample size, is_sumstats, and is_focal |
| `variables` | Character vector of SNP IDs in `chr:pos:ref:alt` format |
| `coef`, `z` | Trait-wise regression coefficients and z-scores (lists with length = number of traits) |

---

#### model_info

Tracks model convergence and optimization metrics for each trait:

| Field | Description |
| --- | --- |
| `model_coveraged` | Whether the joint model converged |
| `outcome_model_coveraged` | Per-trait convergence status |
| `n_updates` | Total number of updates |
| `outcome_n_updates` | Number of updates per trait |
| `profile_loglik` | Log-likelihood per iteration (joint) |
| `outcome_profile_loglik` | Log-likelihood per trait |
| `outcome_proximity_obj` | Proximity statistics per trait per iteration |
| `outcome_coupled_best_update_obj` | Coupled update quality per trait |
| `jk_star` | Jackknife matrix (optional) |

---

#### ucos_details

Describes **uncolocalized effect sets** when no coloc is detected:

| Field | Description |
| --- | --- |
| `ucos_index` | Indices of SNPs in each uncolocalized set |
| `ucos_variables` | SNP IDs in each set |
| `ucos_outcomes` | Trait name associated with each set (e.g., `"Wightman"`) |
| `ucos_weight` | Per-SNP weights in each set |
| `ucos_top_variables` | Top SNP per set |
| `ucos_purity` | Pairwise correlation (min, max, median) across sets |
| `ucos_outcomes_delta` | Delta value (importance) per set |

> Interpretation:
> 
> - Large number of `ucos` suggests GWAS has signals independent of molecular QTLs.
> - `ucos_delta` indicates signal strength; higher = more trait variance explained.

---

#### region_info

| Field | Description |
| --- | --- |
| `region_coord` | Original gene start/end coordinates |
| `grange` | Extended region used for coloc (typically ±1.5Mb) |
| `region_name` | Gene or region ID (can repeat if multiple traits mapped) |

#### computing_time
---


#### Key Results Interpretation in this MWE

#### Analysis Overview

- **Gene**: ENSG00000049246 (chromosome 1)
- **Traits analyzed**:  
  - `trait_A_ENSG00000049246` (xQTL, n = 150)  
  - `Wightman` (GWAS, n = 363,646)
- **Variants**: 6,872 variants analyzed within the region

#### Missing Core Results

- `cos_summary`: `NULL` – No colocalized effects detected  
- `vcp`: `NULL` – No variant colocalization probabilities computed  
- `cos_details`: `NULL` – No detailed colocalization results available

This indicates no significant colocalization between xQTL and GWAS signals for the target gene.

These results suggest that while the Wightman GWAS shows multiple independent association signals in the region, these signals do not colocalize with the xQTL effects. 

### Uncolocalized Effects (`ucos_details`)

The analysis detected 11 uncolocalized effects (`ucos1:y2` to `ucos11:y2`), all associated with the GWAS trait `Wightman`.  

- **Top variants**:  
  - `chr1:9797684:A:C`  
  - `chr1:9809863:C:T`  
  - `chr1:9830837:C:T`  
- **Effect strengths**:  
  - Ranged from extremely weak (`ucos9:y2` with weights ≈ 9e-13)  
  - To moderate (`ucos5:y2` with weights ≈ 3e-06)  
- **Delta values**:  
  - Effect size contributions ranged from 0.0088 to 0.0743

### Model Convergence

- `trait_A_ENSG00000049246`: Converged after 20 updates (`model_coveraged = TRUE`)
- `Wightman`: Did not converge after 501 updates (`model_coveraged = FALSE`)


### Technical Details

- **Analysis region**: chr1:6,120,000–9,960,000  
- **Gene coordinates**: chr1:7,784,319–7,845,176  

**Computing time**:  
- Data loading: ~3 minutes  
- Quality control: ~17 seconds  
- Model fitting:  
  - xQTL: ~15 seconds  
  - GWAS integration: ~4.7 minutes